In [1]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [2]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌅ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌅ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
include("global_adder_passthrough.jl")

filter_highest_tag_number (generic function with 1 method)

In [4]:
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 40
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state)
initial_state_2 = copy(initial_state)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[1, 40, 2, 39, 3, 38, 4, 37, 5, 36, 6, 35, 7, 34, 8, 33, 9, 32, 10, 31, 11, 30, 12, 29, 13, 28, 14, 27, 15, 26, 16, 25, 17, 24, 18, 23, 19, 22, 20, 21]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2]


MPS
[1] ((dim=2|id=684|"Qubit,Site,n=1"), (dim=50|id=932|"Link,l=1"))
[2] ((dim=50|id=932|"Link,l=1"), (dim=2|id=735|"Qubit,Site,n=2"), (dim=50|id=62|"Link,l=2"))
[3] ((dim=50|id=62|"Link,l=2"), (dim=2|id=253|"Qubit,Site,n=3"), (dim=50|id=356|"Link,l=3"))
[4] ((dim=50|id=356|"Link,l=3"), (dim=2|id=787|"Qubit,Site,n=4"), (dim=50|id=300|"Link,l=4"))
[5] ((dim=50|id=300|"Link,l=4"), (dim=2|id=923|"Qubit,Site,n=5"), (dim=50|id=819|"Link,l=5"))
[6] ((dim=50|id=819|"Link,l=5"), (dim=2|id=592|"Qubit,Site,n=6"), (dim=50|id=163|"Link,l=6"))
[7] ((dim=50|id=163|"Link,l=6"), (dim=2|id=520|"Qubit,Site,n=7"), (dim=50|id=49|"Link,l=7"))
[8] ((dim=50|id=49|"Link,l=7"), (dim=2|id=612|"Qubit,Site,n=8"), (dim=50|id=993|"Link,l=8"))
[9] ((dim=50|id=993|"Link,l=8"), (dim=2|id=367|"Qubit,Site,n=9"), (dim=50|id=624|"Link,l=9"))
[10] ((dim=50|id=624|"Link,l=9"), (dim=2|id=8|"Qubit,Site,n=10"), (dim=50|id=751|"Link,l=10"))
[11] ((dim=50|id=751|"Link,l=10"), (dim=2|id=120|"Qubit,Site,n=11"), (dim=50|id=125|"Li

In [5]:
using BenchmarkTools
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
BenchmarkTools.DEFAULT_PARAMETERS.evals = 1
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 300


300

In [6]:
@benchmark initial_state_1 = apply(add1_3,$initial_state_1)


BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  755.511 ms …    2.025 s  ┊ GC (min … max): 14.38% … 25.58%
 Time  (median):     822.043 ms               ┊ GC (median):    13.07%
 Time  (mean ± σ):   839.055 ms ± 125.929 ms  ┊ GC (mean ± σ):  13.43% ±  1.47%

       ▂   █▄█▆▂ ▄▄▂▂▄▂█                                         
  ▆▁▁▆▁█▆▄▆█████████████▄▄██▆▁▄▆▄▁▄▁▁▄▁▁▄▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  756 ms           Histogram: frequency by time          1.01 s <

 Memory estimate: 1.41 GiB, allocs estimate: 49423.

In [ ]:
@benchmark initial_state_2 = global_adder($initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)

In [ ]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds

In [ ]:
println(contract(initial_state_2).tensor)

In [ ]:
# The `seconds` expression helps set a rough time budget, see Manual for more explanation
@btime sin(x) setup=(x=rand()) seconds=3

In [ ]:
# psi = contract(initial_state)
# println(inds(psi))
# print_nonzero_coordinates(psi)

In [ ]:
# function perm(bit_string::String, pattern::Vector{Int})
#     permuted = ['0' for _ in 1:length(bit_string)]
#     for (i, pos) in enumerate(pattern)
#         permuted[pos] = bit_string[i]
#     end
#     return join(permuted)
# end

In [2]:
p_range_str = "0.24210526,0.28421053,0.32631579,0.36842105,0.41052632,0.45263158,0.49473684"
parse.(Float64, split(p_range_str, ","))

7-element Vector{Float64}:
 0.24210526
 0.28421053
 0.32631579
 0.36842105
 0.41052632
 0.45263158
 0.49473684